In [2]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta



In [3]:
x_train = pd.read_csv("outputxNew.csv").to_numpy()
y_train = pd.read_csv("outputyNew.csv").to_numpy()

In [5]:
y_train.shape

(4000, 1)

In [38]:
# tweets = pd.read_csv("tweets.csv")
# tweetI = pd.read_csv("tweet-index.csv")
df = pd.read_csv("AMZN.csv")

In [40]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-01-03,37.896000,37.938000,37.384998,37.683498,37.683498,70422000
1,2017-01-04,37.919498,37.984001,37.709999,37.859001,37.859001,50210000
2,2017-01-05,38.077499,39.119999,38.013000,39.022499,39.022499,116602000
3,2017-01-06,39.118000,39.972000,38.924000,39.799500,39.799500,119724000
4,2017-01-09,39.900002,40.088501,39.588501,39.846001,39.846001,68922000


In [41]:
tweets.post_date = tweets.post_date.astype(str)
i = 0
for tweet in tweets.post_date:
    tweets.loc[i, 'post_date'] = datetime.utcfromtimestamp(int(tweets.loc[i, 'post_date'])).strftime('%Y-%m-%d')
    i = i+1

ValueError: invalid literal for int() with base 10: '2015-01-01'

In [ ]:
merged_df = pd.merge(tweets, tweetI, how='inner', on='tweet_id')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_file.csv', index=False)

In [42]:
mergedFile = pd.read_csv("merged_file.csv")

In [7]:
sentiment_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")


/Users/ashutoshsingh/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [43]:
def flatten(lst):
    """
    Flattens a 2D list into a 1D list.
    """
    flattened = []
    for sublist in lst:
        for item in sublist:
            flattened.append(item)
    return flattened

In [45]:
dates = ["2018-08-30", "2018-08-15","2018-07-30","2018-07-15","2018-06-30","2018-06-15","2018-05-30","2018-05-15","2018-04-30","2018-04-15",
"2017-08-30", "2017-08-15","2017-07-30","2017-07-15","2017-06-30","2017-06-15","2017-05-30","2017-05-15","2017-04-30","2017-04-15"]
x_train = []
y_train = []
for targetDate in dates:
    target_date = pd.to_datetime(targetDate)
    one_week_before = target_date - timedelta(days=7)
    one_week_before = one_week_before.strftime('%Y-%m-%d')
    start_value = one_week_before
    end_value = targetDate
    # Use boolean indexing to select tuples that fall within the specified range
    resultPart = mergedFile[(mergedFile['post_date'] >= start_value) & (mergedFile['post_date'] <= end_value)]
    resultCompany = resultPart[resultPart['ticker_symbol'] == "AMZN"]
    resultCompany = resultCompany.sort_values('retweet_num', ascending=False)
    resultCompany = resultCompany.head(200)
    # tweetsOb = result['body']
    tweetsOb = resultCompany['body'].values
    retweetsOb = resultCompany['retweet_num'].values
    commentOb = resultCompany['comment_num'].values
    likeOb = resultCompany['like_num'].values
    
    daysOb = [(datetime.strptime(targetDate, "%Y-%m-%d")-datetime.strptime(date2, "%Y-%m-%d")).days for date2 in resultCompany['post_date']]
    
    sentiResult = nlp(tweetsOb.tolist())
    labels = [{"positive":0,"neutral":0,"negative":0} for i in sentiResult]
    i =0
    for obj in sentiResult:
        (labels[i])[obj['label']] = obj['score']
        i=i+1
    positive = [elem['positive'] for elem in labels]
    negative = [elem['negative'] for elem in labels]
    neutral = [elem['neutral'] for elem in labels]
    
    labels = [int(i['label'].split()[0]) for i in sentimentResult]
    xComp = [[a,b,c,d,e,f,g] for a,b,c,d,e,f,g in zip(positive, negative, neutral, retweetsOb, commentOb, likeOb, daysOb)]    
#     xComp = flatten(xComp)

    stockTargetDate = target_date + timedelta(days=1)
    six_days_before = stockTargetDate - timedelta(days=6)
    six_days_before = six_days_before.strftime('%Y-%m-%d')

    start_value = six_days_before
    end_value = stockTargetDate.strftime('%Y-%m-%d')

    # Use boolean indexing to select tuples that fall within the specified range
    st = df[(df['Date'] >= start_value) & (df['Date'] <= end_value)]
    diff = st.Close.tolist()[-1] - st.Open.tolist()[0]
    delta = diff / st.Open.tolist()[0]
    delta
    for elem in xComp:
        x_train.append(elem)
        y_train.append(delta)

In [46]:
np.array(x_train).shape

(4000, 7)

In [48]:
np.array(y_train)[:5]

array([0.05102348, 0.05102348, 0.05102348, 0.05102348, 0.05102348])

In [49]:
y_train = np.array(y_train)

In [51]:
y_train = y_train/7

In [53]:
x_train = np.array(x_train)

xDf = pd.DataFrame(x_train, columns=['A', 'B', 'C','D','E','F','G'])
yDf = pd.DataFrame(y_train.tolist(), columns=['values'])

# Export the DataFrame to a CSV file
xDf.to_csv('outputxNew.csv', index=False)
yDf.to_csv('outputyNew.csv', index=False)

In [55]:
xDf

,A,B,C,D,E,F,G
0,0.000000,0.0,0.920105,88.0,4.0,125.0,1.0
1,0.539348,0.0,0.000000,77.0,7.0,321.0,2.0
2,0.000000,0.0,0.903385,59.0,8.0,147.0,1.0
3,0.000000,0.0,0.904519,57.0,27.0,71.0,1.0
4,0.000000,0.0,0.744008,55.0,17.0,73.0,1.0
...,...,...,...,...,...,...,...
3995,0.000000,0.0,0.902423,2.0,1.0,6.0,3.0
3996,0.000000,0.0,0.939989,2.0,0.0,0.0,1.0
3997,0.000000,0.0,0.551905,2.0,1.0,4.0,6.0
3998,0.000000,0.0,0.923244,2.0,0.0,7.0,7.0


In [33]:
my_array = [1, 2, 3, 4, 5]

# Convert array to dataframe
newdf = pd.DataFrame(my_array, columns=['Values'])

# Display dataframe
print(newdf)

   Values
0       1
1       2
2       3
3       4
4       5


In [56]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Define your LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(7, 1)))  # 7 features, 1 time step
model.add(Dense(1))  # Output layer with single node for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Prepare your data
dummyX = x_train
dummyY = y_train
# Reshape x_train to have 3D shape (samples, time steps, features)
dummyX = np.reshape(dummyX, (dummyX.shape[0], dummyX.shape[1], 1))

# Train the LSTM model
model.fit(dummyX, y_train, epochs=100, batch_size=1)

# Once the model is trained, you can use it to make predictions on new data
# Prepare your new data for prediction (assuming x_test is your new data)
# x_test = np.array([[0.0, 0.0, 0.8157535195350647, 27.0, 7.0, 82.0, 1.0],
#                    [0.0, 0.0, 0.9311531782150269, 26.0, 0.0, 26.0, 4.0],
#                    [0.0, 0.0, 0.9164313077926636, 24.0, 0.0, 25.0, 6.0],
#                    [0.0, 0.0, 0.8958992958068848, 23.0, 0.0, 28.0, 6.0],
#                    [0.0, 0.0, 0.8829037547111511, 22.0, 3.0, 115.0, 4.0]])
x_test = x_train.head(20)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
y_pred = model.predict(x_test)



Epoch 1/100


2023-04-08 22:28:05.776347: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


4000/4000 [==============================] - 6s 1ms/step - loss: 3.4653e-04
Epoch 2/100
4000/4000 [==============================] - 5s 1ms/step - loss: 1.0994e-04
Epoch 3/100
4000/4000 [==============================] - 5s 1ms/step - loss: 6.5852e-05
Epoch 4/100
4000/4000 [==============================] - 5s 1ms/step - loss: 5.7517e-05
Epoch 5/100
4000/4000 [==============================] - 5s 1ms/step - loss: 4.6881e-05
Epoch 6/100
4000/4000 [==============================] - 5s 1ms/step - loss: 4.3553e-05
Epoch 7/100
4000/4000 [==============================] - 4s 1ms/step - loss: 4.0463e-05
Epoch 8/100
4000/4000 [==============================] - 5s 1ms/step - loss: 3.7669e-05
Epoch 9/100
4000/4000 [==============================] - 4s 1ms/step - loss: 3.5239e-05
Epoch 10/100
4000/4000 [==============================] - 5s 1ms/step - loss: 3.4465e-05
Epoch 11/100
4000/4000 [==============================] - 5s 1ms/step - loss: 3.3403e-05
Epoch 12/100
4000/4000 [==================

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [66]:
x_test = x_train
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
y_pred = model.predict(x_test)

125/125 [==============================] - 0s 899us/step


In [67]:
y_pred


array([[0.00291349],
       [0.00213732],
       [0.00291355],
       ...,
       [0.00142697],
       [0.00142368],
       [0.0014895 ]], dtype=float32)

In [61]:
y_train[:20]

array([0.00850391, 0.00850391, 0.00850391, 0.00850391, 0.00850391,
       0.00850391, 0.00850391, 0.00850391, 0.00850391, 0.00850391,
       0.00850391, 0.00850391, 0.00850391, 0.00850391, 0.00850391,
       0.00850391, 0.00850391, 0.00850391, 0.00850391, 0.00850391])

In [117]:
def Accuracy(actual,predicted):
    fails = 0;
    passes = 0;
    for i, j in zip(actual,predicted):
        if(i*j<0):
            fails=fails+1
        else:
            passes=passes+1
    return passes/(passes+fails)

# def Accuracy(actual,predicted):
#     sums = 0
#     for i, j in zip(actual,predicted):
#         sums  = sums+abs(i-j)
#     return sums

In [70]:
Accuracy(y_train,y_pred)



array([15.816014], dtype=float32)

In [87]:
y_train = dummyY

newY_train = y_train[::200]

In [100]:
newY_train

(20,)

In [92]:
dummyX = x_train

In [104]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Generate dummy input data with shape (20, 200, 7)
input_data = reshaped_input_data

# Generate dummy target data with shape (20,)
target_data = newY_train

# Define your LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(200, 7)))  # 200 time steps, 7 features
model.add(Dense(1))  # Output layer with single node for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the LSTM model
model.fit(input_data, target_data, epochs=100, batch_size=1)

Epoch 1/100
20/20 [==============================] - 1s 11ms/step - loss: 0.0190
Epoch 2/100
20/20 [==============================] - 0s 11ms/step - loss: 0.0200
Epoch 3/100
20/20 [==============================] - 0s 11ms/step - loss: 0.0057
Epoch 4/100
20/20 [==============================] - 0s 11ms/step - loss: 0.0025
Epoch 5/100
20/20 [==============================] - 0s 11ms/step - loss: 0.0015
Epoch 6/100
20/20 [==============================] - 0s 11ms/step - loss: 0.0015
Epoch 7/100
20/20 [==============================] - 0s 11ms/step - loss: 0.0013
Epoch 8/100
20/20 [==============================] - 0s 11ms/step - loss: 0.0023
Epoch 9/100
20/20 [==============================] - 0s 11ms/step - loss: 4.8678e-04
Epoch 10/100
20/20 [==============================] - 0s 11ms/step - loss: 9.3204e-04
Epoch 11/100
20/20 [==============================] - 0s 11ms/step - loss: 2.7721e-04
Epoch 12/100
20/20 [==============================] - 0s 11ms/step - loss: 3.4560e-04
Epoch 13/

In [121]:
pred = model.predict(reshaped_input_data)

1/1 [==============================] - 0s 30ms/step


In [1]:
Accuracy(pred,newY_train)


NameError: name 'Accuracy' is not defined

In [123]:
newY_train

array([ 0.00850391, -0.00017563, -0.00472493,  0.00804894, -0.00026531,
        0.00341558,  0.00276773, -0.0021967 ,  0.01420438,  0.00110684,
        0.0042887 ,  0.00032094, -0.00807122,  0.00284433, -0.0066931 ,
        0.00356947,  0.00165338,  0.00219718,  0.00756857, -0.0027715 ])

In [124]:
pred

array([[ 1.1451561e-02],
       [ 1.0207165e-02],
       [-4.6559498e-03],
       [ 6.6161640e-03],
       [ 9.9096708e-03],
       [ 1.7795287e-02],
       [ 8.9471526e-03],
       [-2.8177747e-04],
       [ 1.3870038e-02],
       [ 2.0165851e-03],
       [ 8.6123846e-04],
       [-7.7266479e-05],
       [ 1.9711812e-03],
       [ 4.1658692e-03],
       [-2.2750627e-02],
       [ 6.6922791e-03],
       [-2.0249795e-02],
       [ 1.2775488e-02],
       [ 1.1208180e-02],
       [-7.0879720e-03]], dtype=float32)

In [6]:
num_samples = 20
num_timesteps = 200
num_features = 7

reshaped_input_data = np.zeros((num_samples, num_timesteps, num_features))
for i in range(num_samples):
    reshaped_input_data[i] = x_train[i*num_timesteps:(i+1)*num_timesteps]


In [125]:
import pickle


In [126]:
with open('LSTM-Final.pkl', 'wb') as file:
    pickle.dump(model, file)


In [99]:
y_train.shape

(3994,)